# Project notebook

### Importing files

In [74]:
import os
import sys
from pathlib import Path 
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import requests

# np.random.seed(17)

### Data acquisition

In [75]:
load_dotenv()
print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


In [81]:
SYMBOL = 'USDINR'
USE_ALPHA = bool(os.getenv('API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series (Daily)' in k][0]
    df_api = pd.DataFrame(js[key])
    # df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])


In [82]:
df_api

,2025-08-21,2025-08-20,2025-08-19,2025-08-18,2025-08-15,2025-08-14,2025-08-13,2025-08-12,2025-08-11,2025-08-08,...,2025-04-17,2025-04-16,2025-04-15,2025-04-14,2025-04-11,2025-04-10,2025-04-09,2025-04-08,2025-04-07,2025-04-04
1. open,87.0180,87.0430,87.2670,87.5190,87.6870,87.4420,87.5800,87.6463,87.5360,87.4200,...,85.5670,85.6630,86.0010,86.1600,86.1700,86.6160,86.2200,85.8340,85.4820,85.2270
2. high,87.3230,87.1660,87.3720,87.5420,87.6890,87.6800,87.7200,87.7370,87.7350,87.7450,...,85.6125,85.7725,86.0400,86.1600,86.2440,86.6160,86.7100,86.2925,85.8975,85.5930
3. low,86.9210,86.9720,86.9130,87.2760,87.4020,87.3800,87.3710,87.5580,87.4670,87.4200,...,85.2650,85.5310,85.5600,85.8440,85.9100,86.0890,86.2200,85.7870,85.4820,84.9070
4. close,87.2670,86.9980,87.0400,87.2910,87.5080,87.6710,87.4300,87.5860,87.6500,87.4610,...,85.3600,85.5670,85.6630,86.0030,86.1600,86.1700,86.6160,86.2200,85.8160,85.4820
5. volume,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
df_api=df_api.T
df_api=df_api.rename(columns={'1. open':'o','2. high':'h','3. low':'l','4. close':'c','5. volume':'volume'})
df_api=df_api.reset_index()

In [84]:
df_api

,index,o,h,l,c,volume
0,2025-08-21,87.0180,87.3230,86.9210,87.2670,0
1,2025-08-20,87.0430,87.1660,86.9720,86.9980,0
2,2025-08-19,87.2670,87.3720,86.9130,87.0400,0
3,2025-08-18,87.5190,87.5420,87.2760,87.2910,0
4,2025-08-15,87.6870,87.6890,87.4020,87.5080,0
...,...,...,...,...,...,...
95,2025-04-10,86.6160,86.6160,86.0890,86.1700,0
96,2025-04-09,86.2200,86.7100,86.2200,86.6160,0
97,2025-04-08,85.8340,86.2925,85.7870,86.2200,0
98,2025-04-07,85.4820,85.8975,85.4820,85.8160,0


In [85]:
df_api=df_api.rename(columns={'index':'date'})

In [86]:
con_path = Path.cwd().parent / "data" 
con_path=str(con_path) + "/raw"


In [87]:
df_api.to_csv(f'{con_path}/raw.csv')
# con_path

In [88]:
df=pd.read_csv(f'{con_path}/raw.csv')
df

,Unnamed: 0,date,o,h,l,c,volume
0,0,2025-08-21,87.018,87.3230,86.921,87.267,0
1,1,2025-08-20,87.043,87.1660,86.972,86.998,0
2,2,2025-08-19,87.267,87.3720,86.913,87.040,0
3,3,2025-08-18,87.519,87.5420,87.276,87.291,0
4,4,2025-08-15,87.687,87.6890,87.402,87.508,0
...,...,...,...,...,...,...,...
95,95,2025-04-10,86.616,86.6160,86.089,86.170,0
96,96,2025-04-09,86.220,86.7100,86.220,86.616,0
97,97,2025-04-08,85.834,86.2925,85.787,86.220,0
98,98,2025-04-07,85.482,85.8975,85.482,85.816,0


In [92]:
type(df['date'].iloc[1])

str

In [93]:
df['date']=pd.to_datetime(df['date'])

In [94]:
type(df['date'].iloc[1])

pandas._libs.tslibs.timestamps.Timestamp

In [98]:
df=df.drop(df.columns[0], axis=1)

In [101]:
df

,date,o,h,l,c,volume
0,2025-08-21,87.018,87.3230,86.921,87.267,0
1,2025-08-20,87.043,87.1660,86.972,86.998,0
2,2025-08-19,87.267,87.3720,86.913,87.040,0
3,2025-08-18,87.519,87.5420,87.276,87.291,0
4,2025-08-15,87.687,87.6890,87.402,87.508,0
...,...,...,...,...,...,...
95,2025-04-10,86.616,86.6160,86.089,86.170,0
96,2025-04-09,86.220,86.7100,86.220,86.616,0
97,2025-04-08,85.834,86.2925,85.787,86.220,0
98,2025-04-07,85.482,85.8975,85.482,85.816,0


In [100]:
con_path = Path.cwd().parent / "data" 
con_path=str(con_path) + "/processed"

df.to_csv(f'{con_path}/processed.csv')